<a href="https://colab.research.google.com/github/hws2002/MachineLearning_PytorchNScikitLearn/blob/master/chapter4/chapter4_1_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.1.1 테이블 형태 데이터에서 누락된 값 식별

In [ ]:
import pandas as pd
from io import StringIO
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

In [ ]:
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [ ]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [ ]:
# 사이킷런의 추정기에 주입하기 전에 DataFrame의 values 속성을 사용하여 언제나 넘파이 배열을 얻을 수 있음.
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

# 4.1.2 누락된 값이 있는 훈련 샘플이나 특성 제외

In [ ]:
# 누락된 데이터를 다루는 가장 쉬운 방법 중 하나는 데이터셋에서 해당 훈련 샘플(행)이나 특성(열)을 완전히 삭제하는 것임.
# 이는 dropna 메서드를 사용하여 쉽게 삭제 가능
df.dropna(axis = 0)


,A,B,C,D
0,1.0,2.0,3.0,4.0


In [ ]:
# axis 매개변수를 1로 지정하면 열을 삭제
df.dropna(axis = 1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [ ]:
# 모든 열이 NaN일 때만 행을 삭제하기
# (여기에서는 모든 값이 NaN인 행이 없기 때문에 전체 배열이 반환됨.)
df.dropna(how = 'all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [ ]:
# NaN이 아닌 값이 네 개보다 작은 행을 삭제합니다.
df.dropna(thresh = 4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [ ]:
# 특정 열에 NaN이 있는 행만 삭제합니다 (여기서는 C열)
df.dropna(subset = ['C'])

# 특정행에 NaN이 있는 열을 삭제하려면
df.dropna(axis = 1, subset = [2])

,A,B,C
0,1.0,2.0,3.0
1,5.0,6.0,NaN
2,10.0,11.0,12.0


# 4.1.3 누락된 값 대체
종종 훈련 샘플을 삭제하거나 특성 열을 통쨰로 제거하기 어려울 경우에는, 여러 가지 보간 기법을 사용하여 데이터셋에 있는 다른 훈련 샘플로부터 누락된 값을 추정함.

In [ ]:
# 가장 흔한 보간 기법 중 하나는 평균으로 대체하는 것임.
# 각 특성 열의 전체 평균으로 누락된 값을 바꾸는 것.
# 사이킷런의 SimpleImputer 클래스를 사용하자
# strategy 중 most_frequent는 범주형 특성 값을 대체할 때 유용함.
# ex) 빨강, 초록 파랑 같은 색 이름이 인코딩된 특성 열을 다루는 경우
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values = np.nan, strategy = 'mean') # median 이나 most_frequent(mode) 등이 있음
imr = imr.fit(df.values)

In [ ]:
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [ ]:
# 혹은 판다스의 `fillna` 메서드에 매개변수로 누락된 값을 채울 방법을 전달하면 됨.
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


### fillna 메서드 사용하기
`fillna` 메서드의 `method` 매개변수를 사용하여 누락된 값을 채울 수도 있음. `bfill` 또는 `backfill`은 누락된 값을 다음 행의 값으로 채움. `ffill` 또는 `pad`는 누락된 값을 이전 행의 값으로 채움

In [ ]:
df.fillna(method = 'bfill') # method = 'backfill' 과 같음

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


In [ ]:
df.fillna(method='ffill') # method = 'pad'와 같음

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,3.0,8.0
2,10.0,11.0,12.0,8.0


In [ ]:
# axis=1로 지정하면 행이 아니라 열을 사용함
df.fillna(method = 'bfill', axis = 1)

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,8.0,8.0
2,10.0,11.0,12.0,NaN


# 4.1.4 사이킷런 추정기 API 익히기

1. 앞서 사용했던 `SimpleImputer` 클래스는 사이킷런의 **변환기 (transformer)** API의 일부임. 변환기 API를 사용하면 데이터 변환에 관련된 파이썬 클래스를 구현할 수 있음.
이런 추정기의 주요 메서드는 `fit`과 `transform`임.  

2. 3장에서 사용한 분류기는 변환기 API와 매우 유사한 추정기(estimator)임. 추정기는 `predict` 메서드가 있지만 `transform` 메서드도 가질 수 있음.  
분류를 위한 추정기를 훈련할 때 `fit` 메서드를 사용해서 모델의 파라미터를 학습함.   
지도 학습 작업에서는 모델을 훈련할 때 추가적으로 클래스 레이블을 제공함. 그다음 `predict` 메서드를 이용하여 레이블이 없는 새로운 데이터 샘플에 대한 예측을 만듬.